In [1]:
import numpy as np
import pandas as pd

color_est = pd.read_csv("../data/color2emotion_rescaling.csv",index_col = False)
emotion_est = pd.read_csv("../result/emotion_estimation_hand_craft.csv")
action_est = pd.read_csv("../result/action_estimation.csv")

In [2]:
#color_est

In [3]:
emotion_est


,감정,에너지,회복,순환,정화
0,행복,0.096577,0.205271,0.084743,0.154127
1,열정,0.411770,0.417962,0.277941,0.317782
2,두려움,0.064625,0.127702,0.021283,0.112353
3,분노,0.246015,0.288097,0.199031,0.265908
4,우울,0.144277,0.259323,0.124939,0.132359
5,슬픔,0.126782,0.238345,0.068066,0.202515


In [4]:
action_est

,감정,에너지,회복,순환,정화
0,명상,0.142931,0.170405,0.227149,0.216091
1,운동,0.146344,0.134466,0.142635,0.206883
2,산책,0.138340,0.070666,0.036914,0.184480
3,수면,0.143820,0.082785,0.128648,0.185543
4,독서,0.181206,0.096831,0.077269,0.210896


In [65]:
# 감정 -> 효능 <- 해동 시나리오
class emotion2item():
    def __init__(self ,action_df, emotion_df):
        self.action_df = action_df
        self.emothon_df = emotion_df
             
        
    def sim_item(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        MAE_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            MAE = np.mean(np.abs(em_est - act_est))
            
            MAE_dict[act_name] = MAE
            
            print(f'{em_name}과 {act_name}의 MAE : {MAE}')
        
        sorted_dict = sorted(MAE_dict.items(), key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    
            
    def max_item(self, em_idx, n):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        print(em_name,em_est)

    def complete_item(self, em_idx, n):
        pass

In [68]:
XYZ = emotion2item(action_est, emotion_est)
for i in range(0,5):
    XYZ.sim_item(i)
    print('\n')


행복과 명상의 MAE : 0.07139745556438963
행복과 운동의 MAE : 0.057805060253789016
행복과 산책의 MAE : 0.06363757477245398
행복과 수면의 MAE : 0.0612625090094904
행복과 독서의 MAE : 0.06432819407847196
--------------------
행복에서 가장 유사한 item은 운동


열정과 명상의 MAE : 0.16721952789359623
열정과 운동의 MAE : 0.19878148556583455
열정과 산책의 MAE : 0.24876343727939662
열정과 수면의 MAE : 0.22116491281323966
열정과 독서의 MAE : 0.21481323030052918
--------------------
열정에서 가장 유사한 item은 명상


두려움과 명상의 MAE : 0.1076533085045715
두려움과 운동의 MAE : 0.07609135083233315
두려움과 산책의 MAE : 0.05462719484542805
두려움과 수면의 MAE : 0.07616665943836173
두려움과 독서의 MAE : 0.07549530547112226
--------------------
두려움에서 가장 유사한 item은 산책


분노과 명상의 MAE : 0.07467758738332325
분노과 운동의 MAE : 0.09218050560189617
분노과 산책의 MAE : 0.1421624573154582
분노과 수면의 MAE : 0.11456393284930122
분노과 독서의 MAE : 0.10821225033659072
--------------------
분노에서 가장 유사한 item은 명상


우울과 명상의 MAE : 0.06905143109283282
우울과 운동의 MAE : 0.05478571457344862
우울과 산책의 MAE : 0.08368499936269866
우울과 수면의 MAE : 0.0584720425041659
우울과 독

In [8]:
hXYZ = emotion2item(action_est, emotion_est)
hXYZ.sim_item(0)

0.07139745556438963
0.057805060253789016
0.06363757477245398
0.0612625090094904
0.06432819407847196
